In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
from  keras.models import Sequential, Model
from  keras.layers import Input, Dense, LeakyReLU, Activation, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization, Reshape, Lambda

from  keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from  keras.applications.vgg19 import VGG19, preprocess_input
from  keras.applications import ResNet152V2,ResNet101V2
from  keras.applications.resnet_v2 import preprocess_input 

from  keras.optimizers import RMSprop, Adam
from  keras.preprocessing.image import load_img, ImageDataGenerator
from  keras.utils import multi_gpu_model
from IPython.display import Image 

import keras_metrics
import pandas
import ast
import numpy as np
import matplotlib.patches as patches 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw

Using TensorFlow backend.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras.backend as K
import numpy as np
import random

In [4]:
BATCH_SIZE = 64
IMG_SIZE = 224

In [5]:
def yolf_loss(y_true, y_pred):
    #true e pred sono [32][245]
    
    #tensorflow.print("tensors:", y_pred)
    #tensorflow.print("batch_size:", tensorflow.shape(y_pred)[0])
    
    y_pred = tensorflow.reshape(y_pred, [tensorflow.shape(y_pred)[0], 7, 7, 5])
    y_true = tensorflow.reshape(y_true, [tensorflow.shape(y_pred)[0], 7, 7, 5])


    b_p_pred = y_pred[..., 0]
    b_x_pred = y_pred[..., 1]
    b_y_pred = y_pred[..., 2]
    b_w_pred = y_pred[..., 3]
    b_h_pred = y_pred[..., 4]


    b_p = y_true[..., 0]
    b_x = y_true[..., 1]
    b_y = y_true[..., 2]
    b_w = y_true[..., 3]
    b_h = y_true[..., 4]

    #print(b_xy_pred.get_shape(),b_xy.get_shape())
    #print(b_wh_pred.get_shape(),b_wh.get_shape())
    #indicator_coord = K.expand_dims(y_true[..., 3], axis=-1) * 1.0
    loss_p = K.sum(K.square(b_p - b_p_pred), axis=-1)
    loss_xy = K.sum(b_p * (K.square(b_x - b_x_pred) + K.square(b_y - b_y_pred)), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])
    
    b_w = K.pow(b_w, 0.5)
    b_h = K.pow(b_h, 0.5)
    b_w_pred = K.pow(b_w_pred, 0.5)
    b_h_pred = K.pow(b_h_pred, 0.5)
    
    loss_wh = K.sum(
        b_p * 
        (
            (K.square(b_w - b_w_pred)) + 
            (K.square(b_h - b_h_pred))
        ), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])

    #tensorflow.print("loss_p:", loss_p)
    #tensorflow.print("loss_xy:", loss_xy)
    #tensorflow.print("loss_wh:", loss_wh)

    #print(K.cast(loss_p, dtype="float32"))
    #print(K.cast(loss_xy, dtype="float32"))
    #print(loss_wh)
    #tensorflow.print("Loss:", ( K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32") )/3)
    return (K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32"))/3

In [6]:
train_set = "/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/train/"
validation_set = "/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/val/"
test_set = "/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/test/"

In [7]:
train_df = pandas.read_csv("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/train/output_segmentation.csv")
valid_df = pandas.read_csv("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/val/output_segmentation.csv")
test_df = pandas.read_csv("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/test/output_segmentation.csv")

In [8]:
valid_df.columns[1:]

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '235', '236', '237', '238', '239', '240', '241', '242', '243', '244'],
      dtype='object', length=245)

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_set,
        x_col="image_id",
        y_col=train_df.columns[1:],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=validation_set,
        x_col="image_id",
        y_col=valid_df.columns[1:],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

Found 164105 validated image filenames.
Found 18234 validated image filenames.


In [10]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

In [11]:
res = ResNet101V2(include_top=False, weights='imagenet', input_shape = (224,224,3))

In [12]:
res.summary()

Model: "resnet101v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
________________________________________________________________________________________

In [13]:
model = Sequential()
model.add(res)
model.add(Flatten())
model.add(Dense(4096,activation="relu"))
model.add(LeakyReLU(alpha=0.3))
#model.add(Dropout(0.5))
model.add(Dense(245, activation="relu"))
#model.add(Dropout(0.5))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101v2 (Model)          (None, 7, 7, 2048)        42626560  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              411045888 
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 245)               1003765   
Total params: 454,676,213
Trainable params: 454,578,549
Non-trainable params: 97,664
_________________________________________________________________


In [15]:
model = multi_gpu_model(model, gpus=2) #parallelize model

In [16]:
model.compile(optimizer=Adam(lr = 1e-4), loss=yolf_loss, metrics=[yolf_loss])

In [17]:
model.evaluate_generator(valid_generator, steps=step_size_valid, verbose = 1)


285/284 [==============================] - 234s 823ms/step


[22081.798828125, 23005.69921875]

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1, mode='min')
mcp_save = ModelCheckpoint('yolf.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min')

history = model.fit_generator(generator=train_generator, epochs=135, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save, earlyStopping, reduce_lr_loss])

Epoch 1/135
2565/2564 [==============================] - 4428s 2s/step - loss: 560.4275 - yolf_loss: 560.4611 - val_loss: 690.6310 - val_yolf_loss: 575.9771
Epoch 2/135
2565/2564 [==============================] - 4509s 2s/step - loss: 559.3364 - yolf_loss: 559.3531 - val_loss: 626.8984 - val_yolf_loss: 559.1057
Epoch 3/135
2565/2564 [==============================] - 4467s 2s/step - loss: 558.8744 - yolf_loss: 558.8666 - val_loss: 499.5855 - val_yolf_loss: 559.4138
Epoch 4/135
2565/2564 [==============================] - 4486s 2s/step - loss: 558.6618 - yolf_loss: 558.5905 - val_loss: 650.0875 - val_yolf_loss: 557.5112
Epoch 5/135
2565/2564 [==============================] - 4606s 2s/step - loss: 558.3197 - yolf_loss: 558.2963 - val_loss: 495.9128 - val_yolf_loss: 585.2019
Epoch 6/135
2565/2564 [==============================] - 4598s 2s/step - loss: 558.0329 - yolf_loss: 558.0602 - val_loss: 706.5170 - val_yolf_loss: 645.4120
Epoch 7/135
2565/2564 [==============================] - 4

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training accuracy')
plt.plot(val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.ylabel('accuracy')
plt.ylim([min(plt.ylim()),500])
plt.title('Training and Validation accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,20000])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# In[ ]:


print(history.history)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)


test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                    directory=test_set,
                                    x_col="image_id",
                                    y_col=["x_1","y_1","width","height"],
                                    class_mode="raw",
                                    shuffle=True,
                                    target_size=(224, 224),
                                    batch_size=BATCH_SIZE)

STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 


#CHANGE PARALLEL MODEL
pred=model.predict_generator(test_generator,  steps=STEP_SIZE_TEST,  verbose=1)


In [ ]:
print(pred[1])

In [ ]:
# arrange output of pred
min_x, min_y = 100000,100000
max_x, max_y = -1,-1

BLOCK_SIZE = 32
for predict in pred:
    #np arrange qualcosa con 49 x 5
    for el in arranged:
        if el[0] == 1:
            temp_width, temp_height = el[3]*BLOCK_SIZE, el[4]*BLOCK_SIZE,
            temp_lt_x, temp_lt_y = el[1]-temp_width/2, el[2]-temp_heigth/2
            temp_rb_x, temp_rb_y = el[1]-temp_width/2, el[2]-temp_heigth/2
            if temp_lt_x<=min_x and temp_lt_y<=min_y:
                min_x, min_y= temp_lt_x,temp_lt_y
            else if temp_rb_x>=max_x and temp_rb_y>=max_y:
                max_x, max_y= temp_lt_x,temp_lt_y

print(min_x, min_y,max_x, max_y)

In [ ]:
from PIL import  Image, ImageDraw

count = 0

for gen_image in test_generator:
    img = Image.fromarray(np.uint8((gen_image[0][0])*255))
    img1 = ImageDraw.Draw(img)
    x1,y1 = pred[count][0],pred[count][1]
    x4,y4= pred[count][0] + pred[count][2], pred[count][1] + pred[count][3]
    img1.rectangle([(x1,y1),(x4,y4)], outline ="red") 
    #img.show() 
    img.save("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/results/output"+ str(count) + ".jpg")
    #print(gen_image[0][0].shape,pred[count])
    count = count + 1
    if count == 1000:
        break